# Similaridade e Processamento de strings 

## Similaridade de Strings

Aplicações:

* Data Cleaning
* Correção de digitação
* Tradução de idiomas

FuzzyWuzzy - Levenshtein_distance - https://en.wikipedia.org/wiki/Levenshtein_distance

In [1]:
#!pip install fuzzywuzzy

In [2]:
#!pip install fuzzywuzzy[speedup]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 308.4 kB/s eta 0:00:00MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.2-cp39-cp39-linux_x86_64.whl size=80099 sha256=388a8166bac25495558fcc2b2714d4afc540c7ef82feb8f8e01a0315e4952bf2
  Stored in directory: /home/ls/.cache/pip/wheels/46/4a/6c/164a1d9dd67c82d208f19d869ad0a517a0c5a6117f608c53e6
Successfully built python-levenshtein


In [1]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
# Aplicando a fuzzywuzzy em duas strings
s1 = 'Doença Cardiovascular'
s2 = 'Doença Cardiovascular'
fuzz.ratio(s1, s2) # Mede o grau de similaridade

100

In [3]:
s1 = 'Doença Cardiovascular'
s2 = 'Doença Cardiovasculhar'
fuzz.ratio(s1, s2)

98

In [4]:
# Letras maiusculas e minúsculas
s1 = 'Doença Cardiovascular'
s2 = 'doença Cardiovascular'
fuzz.ratio(s1,s2)

95

In [5]:
# Pontuação ou outros caracteres influenciam no score
s1 = 'Doença Cardiovascular'
s2 = 'Doença Cardiovascular!!'
fuzz.ratio(s1,s2)

95

## Similaridade Parcial

* Similaridade parcial busca apenas a string em questão e descarta o resto.
* Extremamente útil para trabalhar com dados coletados da web.

In [6]:
# Consultando o score usando o método ratio
s1 = 'Doença Cardiovascular'
s2 = '###$$%$!Doença Cardiovascular#$#%#ˆˆˆˆˆ!!'
fuzz.ratio(s1, s2)

68

In [7]:
# Consultando o score usando o método partial
s1 = 'Doença Cardiovascular'
s2 = '###$$%$!Doença Cardiovascular#$#%#ˆˆˆˆˆ!!'
fuzz.partial_ratio(s1, s2)

100

In [8]:
# Consultando o score usando o método partial
# alteração nas strings
s1 = 'Doença Cardiovascular'
s2 = '###$$%$!Doença Cardiovasculhar#$#%#ˆˆˆˆˆ!!'
fuzz.partial_ratio(s1, s2)

95

In [9]:
# Ordem de caracteres diferentes?

# Consultando o score usando o método partial
# alteração nas strings
s1 = 'Doença Cardiovascular'
s2 = 'Cardiovascular Doença'
fuzz.partial_ratio(s1, s2)

67

### partial_token_sort_ratio() 

* Função **partial_token_sort_ratio()** separa os tokens por espaço e ordena por ordem alfabética.
* Coloca as strings em letras minúsculas.
* Considera apenas as strings consultadas.

In [10]:
# Consultando o score usando o método partial
# alteração nas strings
s1 = 'Doença Cardiovascular'
s2 = 'Cardiovascular Doença'
fuzz.partial_token_sort_ratio(s1, s2)

100

In [11]:
# Consultando o score usando o método partial
# alteração nas strings
s1 = 'Doença Cardiovascular'
s2 = 'cardiovascular doença'
fuzz.partial_token_sort_ratio(s1, s2)

100

In [12]:
# Consultando o score usando o método partial
# alteração nas strings
s1 = 'Doença Cardiovascular'
s2 = '%%%%%cardiovascular doença&&&&****@@@'
fuzz.partial_token_sort_ratio(s1, s2)

100

In [13]:
fuzz.ratio("Felicidade", "Feliz")

53

## Processando uma Lista de Strings

* Aplicar o fuzzywuzzy para corrigir strings em uma base de dados

In [14]:
from fuzzywuzzy import process

In [15]:
lista = ['Doença Cardiovascular.','doença cardiovascular!!', 'Doenca Cardiovascular', 'Doenc. Cardio']

In [16]:
lista

['Doença Cardiovascular.',
 'doença cardiovascular!!',
 'Doenca Cardiovascular',
 'Doenc. Cardio']

In [17]:
# Extrai os scores de similaridades com uma string em questão
process.extract('Doença Cardiovascular', lista, scorer=fuzz.partial_ratio)

[('Doença Cardiovascular.', 100),
 ('doença cardiovascular!!', 100),
 ('Doenca Cardiovascular', 95),
 ('Doenc. Cardio', 85)]

In [18]:
# Limitando o retorno
process.extract('Doença Cardiovascular', lista, scorer=fuzz.partial_ratio,limit=2)

[('Doença Cardiovascular.', 100), ('doença cardiovascular!!', 100)]

In [19]:
# Método extractOne: Retorna apenas uma string com um score acima de 95
process.extractOne('Doença Cardiovascular', lista, scorer=fuzz.partial_ratio, score_cutoff=95)

('Doença Cardiovascular.', 100)

## Data Cleaning em um DataFrame

* Aplicar o fuzzywuzzy em uma base de dados
* Medir a similaridade de strings e fazer **Data Cleaning**

In [20]:
import pandas as pd
from collections import OrderedDict
data = OrderedDict(
    {
        'codigo_produto': [10 , 11, 12, 13, 14],
        'descrição': ['iphone 6ss', 'iphone 6s', 'iphoni 6s', 'ipone 6s','Iphone 6s,,,,']
        
    })

In [21]:
data

OrderedDict([('codigo_produto', [10, 11, 12, 13, 14]),
             ('descrição',
              ['iphone 6ss',
               'iphone 6s',
               'iphoni 6s',
               'ipone 6s',
               'Iphone 6s,,,,'])])

In [22]:
dataset = pd.DataFrame(data)

In [23]:
dataset

,codigo_produto,descrição
0,10,iphone 6ss
1,11,iphone 6s
2,12,iphoni 6s
3,13,ipone 6s
4,14,"Iphone 6s,,,,"


In [24]:
process.extractOne('Iphone 6s', choices=dataset.descrição, scorer=fuzz.ratio, score_cutoff=95)

('iphone 6s', 100, 1)

In [25]:
def AplicaFuzzy(query, dados, metodo_ratio, score_corte):
    return process.extractOne(query, choices=dados, scorer=metodo_ratio, score_cutoff=score_corte)

In [26]:
AplicaFuzzy('Iphone 6s', dataset.descrição, fuzz.ratio, 90)

('iphone 6s', 100, 1)

**Criando uma nova coluna no Dataset a partir das string similares**

In [27]:
dataset['descrição2'] = AplicaFuzzy('Iphone 6s', dataset.descrição, fuzz.ratio, 95)[0]

In [28]:
dataset

,codigo_produto,descrição,descrição2
0,10,iphone 6ss,iphone 6s
1,11,iphone 6s,iphone 6s
2,12,iphoni 6s,iphone 6s
3,13,ipone 6s,iphone 6s
4,14,"Iphone 6s,,,,",iphone 6s
